In [1]:
from functions import get_points, write_to_laz #, recursive_plane

import pandas as pd
from pyntcloud import PyntCloud
import numpy as np
import pdal
import shapely.wkt
from numpy.lib import recfunctions as rfn

import numpy as np
import pandas as pd
import matplotlib
pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from pyntcloud import PyntCloud
from sklearn.cluster import DBSCAN

from scipy.spatial import cKDTree

from pyntcloud.ransac.fitters import single_fit
from pyntcloud.ransac.models import RansacPlane

In [2]:
READ_PIPELINE = """
{{
    "pipeline": [
        {{
            "type": "readers.ept",
            "filename": "{path}",
            "bounds": "{bounds}"
        }},
        {{
            "type":"filters.crop",
            "polygon":"{wkt}"
        }}
    ]
}}
"""

WRITE_PIPELINE = """
{{
    "pipeline": [
        {{
            "type": "writers.las",
            "filename": "{path}",
            "extra_dims": "all"
        }}
    ]
}}
"""


def get_points(ept_path, bounds, wkt):
    pipeline = pdal.Pipeline(READ_PIPELINE.format(
        path=ept_path,
        bounds=bounds,
        wkt=wkt)
    )

    pipeline.validate()
    pipeline.execute()
    point_cloud = pipeline.arrays[0]

    return point_cloud


def write_to_laz(point_cloud, path):
    pipeline = pdal.Pipeline(
        WRITE_PIPELINE.format(path=path),
        arrays=[point_cloud]
    )
    pipeline.validate()
    pipeline.execute()


def hausdorff_distance(reference_cloud, compared_cloud):
    """

    Parameters
    ----------
    reference_cloud : (Mx3) array
        The X, Y, Z coordinates of points in the reference cloud
    compared_cloud : (Mx3) array
        The X, Y, Z coordinates of points in the compared cloud
    """
    tree = cKDTree(reference_cloud)
    distances, _ = tree.query(compared_cloud, k=1)
    return distances

def filter_distance(reference_cloud, compared_cloud, max_dist = 0.05):
    reference_array = np.array(reference_cloud[['X', 'Y', 'Z']].tolist())
    compared_array = np.array(compared_cloud[['X', 'Y', 'Z']].tolist())

    haus_distances = hausdorff_distance(reference_array, 
                                        compared_array)
    out_array = rfn.append_fields(compared_cloud[['X','Y','Z','Red','Green','Blue']], 
                                'Distance', 
                                haus_distances)
    filtered_out_array = out_array[out_array['Distance'] < max_dist]

    return filtered_out_array





In [7]:
# test_area = [117813.6,487266.1,117942.1,487370.7]
# xmin,ymin,xmax,ymax = test_area

path_a10_2018 = '/var/data/rws/data/2018/entwined/ept.json'
path_a10_2019 = '/var/data/rws/data/2019/amsterdam_entwined/ept.json'

# random area: 
# wkt = 'POLYGON((117813.6 487266.1,117813.6 487370.7,117942.1 487370.7,117942.1 487266.1,117813.6 487266.1))'

# small area: 
# wkt = 'POLYGON((117813 487266,117813 487270,117815 487270,117815 487266,117813 487266))' # small! 

# portal:
# wkt = 'POLYGON((118032.6 488782.8,118032.6 488793.5,118078.8 488793.5,118078.8 488782.8,118032.6 488782.8))' # portal

# portal zonder palen
# wkt = 'POLYGON((118037.0 488782.8,118037.0 488793.5,118072.0 488793.5,118072.0 488782.8,118037.0 488782.8))' # portal zonder palen

# brug
wkt = 'POLYGON((117875.3 487304,117875.3 487329,117906.1 487329,117906.1 487304,117875.3 487304))' # brug

# stukje brug
# wkt = 'POLYGON((117890.1 487311,117890.1 487312.5,117892 487312.5,117892 487311,117890.1 487311))' # brug


xmin,ymin,xmax,ymax = shapely.wkt.loads(wkt).bounds

bounds = f'([{xmin}, {xmax}], [{ymin}, {ymax}])'
pc18 = get_points(path_a10_2018, bounds, wkt)
print(f'loaded pc 2018')

bounds = f'([{xmin}, {xmax}], [{ymin}, {ymax}])'
pc19 = get_points(path_a10_2019, bounds, wkt)
print(f'loaded pc 2019')

loaded pc 2018
loaded pc 2019


In [8]:
filtered_2018 = filter_distance(pc18, pc19)
print('filtered')
points_2018 = pd.DataFrame(
    np.array(filtered_2018[['X','Y','Z','Red','Green','Blue']].tolist()), 
    columns=['x', 'y', 'z','red','green','blue'])
print('dataframed')

filtered
dataframed


In [12]:
try:
    del points_with_planes
except:
    pass
tst = recursive_planes(points_2018.copy(), 2, 10000)
print('fit planes')

ransac = rfn.append_fields(filtered_2018[['X','Y','Z','Red','Green','Blue']], 
                              'Classification', 
                              tst['cid'].astype(np.uint16),
                              usemask=False
                             )

write_to_laz(ransac, '/var/data/rws/data/output/tst_ransac/brug_planes.laz')
print('wrote')



[ 1.17892977e+05  4.87315606e+05 -1.27912968e-01]
[ 0.0311825  -0.01177057  0.9994444 ]
[ 1.17892518e+05  4.87315549e+05 -1.17779567e-01]
[ 0.02886766 -0.01166215  0.99951521]
[ 1.17892385e+05  4.87317331e+05 -8.92014563e-02]
[ 0.0283879  -0.01215366  0.99952309]
[ 1.17892167e+05  4.87316490e+05 -9.36924494e-02]
[ 0.02795337 -0.01177476  0.99953988]
fit planes
wrote


In [14]:
np.unique(ransac['Classification'], return_counts=True)

(array([ 1,  2,  3,  4, 10], dtype=uint16),
 array([  1735,   1615,   1648,   1629, 989422]))